In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import os
import tempfile
from collections import defaultdict

import numpy as np
import tensorflow as tf
import tensorflow.keras.backend as K

from tensorflow import keras
from tensorflow.keras.layers import *
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import save_model, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import custom_object_scope

In [4]:
from qkeras import *
from qkeras import QActivation
from qkeras import QDense
from qkeras import QConv2D
from qkeras import quantized_bits, quantized_relu
from qkeras.utils import load_qmodel
from qkeras.utils import print_model_sparsity
from qkeras.utils import model_save_quantized_weights

In [5]:
##FOr data from idx file_byte
import struct
def load_idx_file(file_path):
    """
    Load data from an IDX file (either images or labels).
    Returns:
        numpy.ndarray: Loaded data as a NumPy array.
    """
    with open(file_path, 'rb') as f:
        # Read the magic number and dimensions
        magic, = struct.unpack('>I', f.read(4))
        if magic == 0x00000801:  # Labels
            num_items, = struct.unpack('>I', f.read(4))
            data = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_items)
        elif magic == 0x00000803:  # Images
            num_items, rows, cols = struct.unpack('>III', f.read(12))
            data = np.frombuffer(f.read(), dtype=np.uint8).reshape(num_items, rows, cols)
        else:
            raise ValueError(f"Unknown magic number: {magic}")
    return data

In [18]:
NB_EPOCH = 20
BATCH_SIZE = 64
VERBOSE = 1
NB_CLASSES = 47
VALIDATION_SPLIT = 0.1
OPTIMIZER = Adam(learning_rate=0.001)
#OPTIMIZER = SGD(learning_rate=0.001, momentum=0.9)

W = 4  #QConv2d W_bit 8
I = 0  # fractional bits
Z = float(4)  #QActivation quantize_bit

In [8]:
from tensorflow.keras.callbacks import LearningRateScheduler

def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    return lr * tf.math.exp(-0.1)

CALLBACKS = lr_schedule = LearningRateScheduler(scheduler)

In [19]:
model = Sequential([
    # Flatten input (28x28 grayscale images for EMNIST)
    Flatten(input_shape=(28, 28)),
    
    # First dense layer with quantized weights and activations
    QDense(
        128,  # Number of neurons
        kernel_quantizer=quantized_bits(W, I, 1),  # W-bit weights with I fractional bits
        bias_quantizer=quantized_bits(W, I, 1),    # W-bit biases
        name="qdense_1"
    ),
    QActivation("quantized_relu(6,2)", name="qact_1"),  # 6-bit activation with 2 fractional bits
    
    # Second dense layer
    QDense(
        64,  # Reduced neurons for lower complexity
        kernel_quantizer=quantized_bits(W, I, 1),
        bias_quantizer=quantized_bits(W, I, 1),
        name="qdense_2"
    ),
    QActivation("quantized_relu(6,2)", name="qact_2"),
    
    # Output layer
    QDense(
        47,  # EMNIST Balanced has 47 classes
        kernel_quantizer=quantized_bits(W, I, 1),
        bias_quantizer=quantized_bits(W, I, 1),
        name="qoutput"
    ),
    Activation("softmax", name="Softmax")
])

In [5]:
def build_model(input_shape):
    x = x_in = Input(shape=input_shape, name="input_layer")
    a = QConv2D(
        32, (2, 2), strides=(2, 2),
        kernel_quantizer=quantized_bits(W, I, 1),
        bias_quantizer=quantized_bits(W, I, 1),
        name="conv2d_L1")(x)
    b = QActivation("quantized_relu(4, 0)", name="activation_1")(a)
    c = QConv2D(
        64, (3, 3), strides=(2, 2),
        kernel_quantizer=quantized_bits(W, I, 1),
        bias_quantizer=quantized_bits(W, I, 1),
        name="conv2d_L2")(b)
    d = QActivation("quantized_relu(4, 0)", name="activation_2")(c)
    e = QConv2D(
        64, (2, 2), strides=(2, 2),
        kernel_quantizer=quantized_bits(W, I, 1),
        bias_quantizer=quantized_bits(W, I, 1),
        name="conv2d_L3")(d)
    f = QActivation("quantized_relu(4, 0)", name="activation_3")(e)
    g = Flatten()(f)
    h = QDense(NB_CLASSES, kernel_quantizer=quantized_bits(W, I, 1),
                   bias_quantizer=quantized_bits(W, I, 1),
                   name="dense")(g)
    i = Activation("softmax", name="Softmax")(h)

    model = Model(inputs=[x_in], outputs=[i])
    return model

In [20]:
def train_and_save(model, x_train, y_train, x_test, y_test):
    model.compile(
        loss="categorical_crossentropy",
       # loss="sparse_categorical_crossentropy",
     #   loss= keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer= OPTIMIZER,
        metrics=["accuracy"])
    
    # Print the model summary.
    model.summary()
    
    model.fit(
        x_train,
        y_train,
        batch_size=BATCH_SIZE,
        epochs=NB_EPOCH,
        validation_split=VALIDATION_SPLIT,
        verbose=VERBOSE,
        callbacks=[CALLBACKS],
        validation_data=(x_test, y_test))
    
    score = model.evaluate(x_test, y_test, verbose=0)
    print("Test loss:", score[0])
    print("Test accuracy:", score[1])

    print_model_sparsity(model)
    
    # Export and import the model. Check that accuracy persists.
    _, keras_file = tempfile.mkstemp(suffix=".keras")  # Ensure the suffix is .keras
    print("Saving model to:", keras_file)
    # Save the model in the new .keras format
    #save_model(model, keras_file)
    model.save("modelx.keras", save_format="keras")
    print("Model saved to modelx.keras")
    print_qstats(model)

In [12]:
image_file_train = "/home/mindspore/work/emnist-balanced-train-images-idx3-ubyte"
label_file_train = "/home/mindspore/work/emnist-balanced-train-labels-idx1-ubyte"

image_file_test = "/home/mindspore/work/emnist-balanced-test-images-idx3-ubyte"
label_file_test = "/home/mindspore/work/emnist-balanced-test-labels-idx1-ubyte"

x_train = load_idx_file(image_file_train)  # Load train images
y_train = load_idx_file(label_file_train)  # Load train labels

x_test = load_idx_file(image_file_test)  # Load test images
y_test = load_idx_file(label_file_test)  # Load test labels

print("Images shape:", x_train.shape)
print("Labels shape:", y_train.shape)
print("Images shape:", x_test.shape)
print("Labels shape:", y_test.shape)

Images shape: (112800, 28, 28)
Labels shape: (112800,)
Images shape: (18800, 28, 28)
Labels shape: (18800,)


In [13]:
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")
x_train /= 255
x_test /= 255
print("x_train shape:", x_train.shape)
print(x_train.shape[0], "train samples")
print(x_test.shape[0], "test samples")

# convert class vectors to binary class matrices //one-hot key 0-9 class matrice lable 0/1
y_train = to_categorical(y_train, NB_CLASSES)
y_test = to_categorical(y_test, NB_CLASSES)

x_train shape: (112800, 28, 28)
112800 train samples
18800 test samples


In [ ]:
#train model and save data

train_and_save(model, x_train, y_train, x_test, y_test)



In [ ]:
print("Reloading model")
    
with custom_object_scope({'TFOpLambda': Lambda}):
        loaded_model = load_qmodel('modelx.keras')

loaded_model.summary()  

score = model.evaluate(x_test, y_test, verbose=0)
print("Test loss:", score[0])
print("Test accuracy:", score[1])